### Install dependencies

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

### Setting up the environment variable

In [ ]:
import os

In [ ]:
PIPELINE_NAME="kfp_tfx_pipeline"
REGION="us-central1"
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"kfp-tfx",PIPELINE_NAME)
print(PROJECT_DIR)

In [ ]:
# Read GCP project id from environment.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

In [ ]:
# Docker image name for the pipeline image
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/tfx-kfp-pipeline'
# please provide the kubeflow pipeline URI
ENDPOINT=""
if ENDPOINT=="":
    raise "Please provide ENDPOINT."

In [ ]:
# verify the pipeline_name and endpoint_name
!echo $PIPELINE_NAME
!echo $ENDPOINT

####  `NOTE` : Go to `kubeflow_runner.py` file & make changes `PROJECT_NAME` & `PIPELINE_NAME`

In [ ]:
# Change the current working directory to project directory 
%cd {PROJECT_DIR}

In [ ]:
# confirm the working directory
!pwd

### Create GCS bucket

In [ ]:
!gsutil mb -l $REGION gs://$GCP_PROJECT_ID-kubeflowpipelines-default

In [ ]:
# copy data to GCS bucket. You can configure the path
!gsutil cp data/data.csv gs://$GCP_PROJECT_ID-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

In [ ]:
# TFX pipeline command to create docker build and push to kubeflow endpoint 
!tfx pipeline create  \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-image

### Run the pipeline on kubeflow

In [ ]:
!tfx run create --pipeline-name=$PIPELINE_NAME --endpoint=$ENDPOINT

### Update the pipeline
! tfx pipeline update \
  --pipeline-path=kubeflow_runner.py \
  --endpoint={ENDPOINT}

! tfx run create --pipeline-name "{PIPELINE_NAME}"